## Library importation 

In [27]:
import datetime
import os
import pandas as pd
import numpy as np
from pprint import pprint
import requests
# Scrappings tools for downloading
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Set a time tracker 
We use it to count a global time for our script execution

In [3]:
start = time.time()

### Initialize Selenium webdriver for Chrome
As we are using Selenium in a remote Notebook, we don't have a real OS for testing purpose. So Selenium has a feature that help us to make web scrapping from a remote server.
The feature is a Remote driver.
The url of our remote driver online tool manager (**Selenium grid**) is <a target="_blank" href="http://141.145.214.205:4444">here</a>


In [4]:
# Setting options for the web driver 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
hub_url = "http://selenium-hub:4444/wd/hub"

## 1. Creation of a function that return Qualifiyings links 
It is important to run Selenium driver with **with** expression as in the next lines, is a best practice that help us to preserve our memory by closing the browser usage in memory, even if we have an error that occurs when the scripts has started

In [5]:

def get_round_urls():
    round_link_by_page = []

    # Calling remote driver for selenium
    with webdriver.Remote(command_executor=hub_url, options=chrome_options) as driver:
        for page in range(1, 12):
            i = 1
            round_urls = []
            base_url = "https://www.f1-fansite.com/tag/f1-qualifying-times/page/%s/" % page
        
            # Open the web page
            driver.get(base_url)

            # Now grab the all rounds html link elements by their XPATH
            round_elements_list = driver.find_elements(By.XPATH,'//div[@class="post_content pc_full"]/h2/a')

            if(len(round_elements_list) > 0):
                for round in round_elements_list:
                    round_urls.append(round.get_attribute("href"))

            # Create a dictionnary that contains all the rounds for a year
            value = {
                "page": page,
                "rounds": round_urls
            }
        
            round_link_by_page.append(value)

            # As we are in the same navigation session, we need to change it after the 
            # first iteration to avoid to try to accept the cookie again. 
            i += 1 
    
        driver.close()
        
    return round_link_by_page

In [6]:
urls = get_round_urls()

In [7]:
end = time.time()

In [8]:
str(datetime.timedelta(seconds=(end - start)))

'0:00:11.824815'

In [16]:
urls[0]["rounds"][0]

'https://www.f1-fansite.com/f1-result/qualifying-results-2023-belgian-f1-sprint-shootout/'

In [31]:
with webdriver.Remote(command_executor=hub_url, options=chrome_options) as driver:

    base_url = urls[0]["rounds"][9]
    
    # Open the web page
    driver.get(base_url)
    
    # Now grab the all rounds html link elements by their XPATH
    title = driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[1]/h1')
    weather = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[4]/div/div[1]/div[1]/div[2]/p[2]')
    print(title.text)
    print(weather.text)
    v = weather.text
    t = title.text
    
    
    
    driver.close()

Qualifying Results 2023 Monaco F1 GP
Weather: dry  26°C
Tarmac: dry  48°C
Humidity: 33%
Wind: 1 km/h W
Pressure: 1016 bar


In [32]:
pattern = r'\b\d{4}\b'
matches = re.findall(pattern, t)

if matches:
    year = matches[0]
    print(year)

2023


In [24]:
v.split("\n")

['Weather: dry  18°C',
 'Tarmac: dry  25°C',
 'Humidity: 86%',
 'Wind: 1 km/h SW',
 'Pressure: 962 mbar']